In [1]:
import numpy as np
import pandas as pd
import datetime
from pytz import timezone
import json

In [2]:
class myClass():
    
    noCandles = 2000
    
    pars = {'loadMethod' : None,
            'dbName' : None,
            'fileName' : None,
            'loadFuntion': None}
    
    @classmethod
    def reset(cls):
        """Reset class parameters"""
        for par in cls.pars:
            cls.pars[par] = None

    @classmethod
    def init(cls, string):
        
        cls.reset()
        
        if string['loadMethod'] == 'influxdb':
            cls.pars['loadMethod'] = string['loadMethod']
            cls.pars['dbName'] = string['dbName']
            cls.pars['loadFunction'] = string['loadFunction']
            
        elif string['loadMethod'] == 'file':
            cls.pars['loadMethod'] = string['loadMethod']
            cls.pars['fileName'] = string['fileName']
            cls.pars['loadFunction'] = string['loadFunction']
            
        else:
            raise ValueError

In [3]:
def fun(a, b, client):
    print('execute fun')
    print(client)
    
myClass.init({'loadMethod': 'influxdb', 'dbName': 'name1', 'loadFunction': fun})

In [30]:
myClass.pars
keyargs = {'client': 3}
myClass.pars['loadFunction'](1,2, **keyargs)

execute fun
3


In [4]:
myClass.init({'loadMethod': 'file', 'fileName': 'name2'})

In [5]:
myClass.pars

{'loadMethod': 'file', 'dbName': None, 'fileName': 'name2'}

In [7]:
import json
json.dumps('some text goes here')

'"some text goes here"'

In [32]:
def fun1():
    print('executing fun1')

def fun2():
    return fun1()

fun2()

executing fun1


In [4]:
from influxdb import DataFrameClient
def loadDataFromInfluxdb(dtLimit, direction, client, noCandles):
    """Actual implementation of data loading from Influxdb"""
    if direction == 'left':
        query = 'SELECT "time", "open", "high", "low", "close" FROM "rates" WHERE time < \'%s\' ORDER BY DESC LIMIT %i' % (dtLimit, noCandles)
    
    elif direction == 'right':
        query = 'SELECT "time", "open", "high", "low", "close" FROM "rates" WHERE time > \'%s\' ORDER BY ASC LIMIT %i' % (dtLimit, noCandles)
    
    else: raise Exception('Direction unknown during loading new data!')
    
    try:
        rates = client.query(query)['rates']
    except KeyError:
        raise Exception('No such data exist in database!')
    
    # Move date indices to new column
    rates['Date'] = rates.index
    # Reset index to numerical one
    rates = rates.reset_index(drop=True)
    # Move Date column in front
    rates = rates.loc[:, ['Date', 'open', 'high', 'low', 'close']]
    # Rename other columns
    rates.columns = ['Date', 'Open', 'High', 'Low', 'Close']
    
    return rates


In [13]:
import pandas as pd
container = pd.DataFrame(columns=['Date', 'Open', 'High', 'Low', 'Close'])

In [65]:
print(container)

Empty DataFrame
Columns: [Date, Open, High, Low, Close]
Index: []


In [28]:
client = DataFrameClient(host='127.0.0.1', port=8086, database='oanda.eurusd.M1')
rates = loadDataFromInfluxdb('2019-06-10T22:55:00Z', 'right', client, 30)

In [29]:
print(rates)

                        Date     Open     High      Low    Close
0  2019-06-10 22:56:00+00:00  1.13111  1.13113  1.13106  1.13107
1  2019-06-10 22:57:00+00:00  1.13108  1.13113  1.13108  1.13112
2  2019-06-10 22:58:00+00:00  1.13109  1.13110  1.13107  1.13108
3  2019-06-10 22:59:00+00:00  1.13103  1.13105  1.13103  1.13105
4  2019-06-10 23:00:00+00:00  0.00000  0.00000  0.00000  0.00000
5  2019-06-10 23:01:00+00:00  1.13107  1.13107  1.13107  1.13107
6  2019-06-10 23:02:00+00:00  1.13107  1.13107  1.13107  1.13107
7  2019-06-10 23:03:00+00:00  1.13108  1.13108  1.13108  1.13108
8  2019-06-10 23:04:00+00:00  1.13109  1.13109  1.13109  1.13109
9  2019-06-10 23:05:00+00:00  1.13110  1.13110  1.13110  1.13110
10 2019-06-10 23:06:00+00:00  1.13105  1.13110  1.13105  1.13108
11 2019-06-10 23:07:00+00:00  1.13109  1.13117  1.13105  1.13117
12 2019-06-10 23:08:00+00:00  1.13118  1.13118  1.13118  1.13118
13 2019-06-10 23:09:00+00:00  1.13118  1.13121  1.13116  1.13116
14 2019-06-10 23:10:00+00

In [26]:
rates.replace(0.0, np.nan, inplace=True)`

In [27]:
rates.to_json(date_format='iso', orient='records')

'[{"Date":"2011-01-03T23:01:00.000Z","Open":1.33594,"High":1.33594,"Low":1.33593,"Close":1.33593},{"Date":"2011-01-03T23:02:00.000Z","Open":1.33595,"High":1.33607,"Low":1.33595,"Close":1.33607},{"Date":"2011-01-03T23:03:00.000Z","Open":1.33605,"High":1.33624,"Low":1.33601,"Close":1.33613},{"Date":"2011-01-03T23:04:00.000Z","Open":1.33611,"High":1.33611,"Low":1.33611,"Close":1.33611},{"Date":"2011-01-03T23:05:00.000Z","Open":null,"High":null,"Low":null,"Close":null},{"Date":"2011-01-03T23:06:00.000Z","Open":1.33608,"High":1.33608,"Low":1.33597,"Close":1.336},{"Date":"2011-01-03T23:07:00.000Z","Open":1.33602,"High":1.33605,"Low":1.33602,"Close":1.33605},{"Date":"2011-01-03T23:08:00.000Z","Open":1.33607,"High":1.33609,"Low":1.33606,"Close":1.33606},{"Date":"2011-01-03T23:09:00.000Z","Open":1.33604,"High":1.33604,"Low":1.33588,"Close":1.33588},{"Date":"2011-01-03T23:10:00.000Z","Open":1.33586,"High":1.33586,"Low":1.33581,"Close":1.33584},{"Date":"2011-01-03T23:11:00.000Z","Open":1.33586,"H

In [5]:
import json
print(json.dumps(rates))

TypeError: Object of type DataFrame is not JSON serializable

In [15]:
print(type(rates))
print(type(rates) == pd.DataFrame)

<class 'pandas.core.frame.DataFrame'>
True


In [7]:
ratesJson = rates.to_json(date_format='iso', orient='records')
print(json.dumps(ratesJson))

"[{\"Date\":\"2012-01-03T13:43:00.000Z\",\"Open\":1.30274,\"High\":1.30299,\"Low\":1.30274,\"Close\":1.30299},{\"Date\":\"2012-01-03T13:44:00.000Z\",\"Open\":1.30297,\"High\":1.30297,\"Low\":1.30274,\"Close\":1.30291},{\"Date\":\"2012-01-03T13:45:00.000Z\",\"Open\":1.30288,\"High\":1.30302,\"Low\":1.30253,\"Close\":1.30267},{\"Date\":\"2012-01-03T13:46:00.000Z\",\"Open\":1.30264,\"High\":1.30306,\"Low\":1.30262,\"Close\":1.30306},{\"Date\":\"2012-01-03T13:47:00.000Z\",\"Open\":1.30309,\"High\":1.30309,\"Low\":1.30304,\"Close\":1.30304},{\"Date\":\"2012-01-03T13:48:00.000Z\",\"Open\":1.30306,\"High\":1.30309,\"Low\":1.30301,\"Close\":1.30307},{\"Date\":\"2012-01-03T13:49:00.000Z\",\"Open\":1.30304,\"High\":1.30318,\"Low\":1.30304,\"Close\":1.30304},{\"Date\":\"2012-01-03T13:50:00.000Z\",\"Open\":1.30301,\"High\":1.30306,\"Low\":1.30293,\"Close\":1.30298},{\"Date\":\"2012-01-03T13:51:00.000Z\",\"Open\":1.30296,\"High\":1.30298,\"Low\":1.30281,\"Close\":1.30287},{\"Date\":\"2012-01-03T13:

In [52]:
rates2 = loadDataFromInfluxdb('2011-01-04T23:02:00Z', 'right', client, 2000)

In [60]:
print(type(rates2))

<class 'pandas.core.frame.DataFrame'>


In [66]:
container = pd.concat([container, rates])
print(container)

                           Date     Open     High      Low    Close
0     2012-01-03 13:43:00+00:00  1.30274  1.30299  1.30274  1.30299
1     2012-01-03 13:44:00+00:00  1.30297  1.30297  1.30274  1.30291
2     2012-01-03 13:45:00+00:00  1.30288  1.30302  1.30253  1.30267
3     2012-01-03 13:46:00+00:00  1.30264  1.30306  1.30262  1.30306
4     2012-01-03 13:47:00+00:00  1.30309  1.30309  1.30304  1.30304
...                         ...      ...      ...      ...      ...
1995  2012-01-04 22:58:00+00:00  1.29454  1.29454  1.29440  1.29452
1996  2012-01-04 22:59:00+00:00  1.29450  1.29453  1.29419  1.29419
1997  2012-01-04 23:00:00+00:00  1.29417  1.29417  1.29389  1.29414
1998  2012-01-04 23:01:00+00:00  1.29412  1.29412  1.29398  1.29404
1999  2012-01-04 23:02:00+00:00  1.29406  1.29407  1.29386  1.29396

[2000 rows x 5 columns]


In [67]:
container = pd.concat([container, rates2])
print(container)

                           Date     Open     High      Low    Close
0     2012-01-03 13:43:00+00:00  1.30274  1.30299  1.30274  1.30299
1     2012-01-03 13:44:00+00:00  1.30297  1.30297  1.30274  1.30291
2     2012-01-03 13:45:00+00:00  1.30288  1.30302  1.30253  1.30267
3     2012-01-03 13:46:00+00:00  1.30264  1.30306  1.30262  1.30306
4     2012-01-03 13:47:00+00:00  1.30309  1.30309  1.30304  1.30304
...                         ...      ...      ...      ...      ...
1995  2011-01-06 08:28:00+00:00  1.31464  1.31475  1.31453  1.31475
1996  2011-01-06 08:29:00+00:00  1.31473  1.31515  1.31473  1.31512
1997  2011-01-06 08:30:00+00:00  1.31515  1.31531  1.31498  1.31509
1998  2011-01-06 08:31:00+00:00  1.31507  1.31525  1.31498  1.31522
1999  2011-01-06 08:32:00+00:00  1.31520  1.31528  1.31484  1.31484

[4000 rows x 5 columns]


In [68]:
print(container.to_json(date_format='iso', orient='records'))

[{"Date":"2012-01-03T13:43:00.000Z","Open":1.30274,"High":1.30299,"Low":1.30274,"Close":1.30299},{"Date":"2012-01-03T13:44:00.000Z","Open":1.30297,"High":1.30297,"Low":1.30274,"Close":1.30291},{"Date":"2012-01-03T13:45:00.000Z","Open":1.30288,"High":1.30302,"Low":1.30253,"Close":1.30267},{"Date":"2012-01-03T13:46:00.000Z","Open":1.30264,"High":1.30306,"Low":1.30262,"Close":1.30306},{"Date":"2012-01-03T13:47:00.000Z","Open":1.30309,"High":1.30309,"Low":1.30304,"Close":1.30304},{"Date":"2012-01-03T13:48:00.000Z","Open":1.30306,"High":1.30309,"Low":1.30301,"Close":1.30307},{"Date":"2012-01-03T13:49:00.000Z","Open":1.30304,"High":1.30318,"Low":1.30304,"Close":1.30304},{"Date":"2012-01-03T13:50:00.000Z","Open":1.30301,"High":1.30306,"Low":1.30293,"Close":1.30298},{"Date":"2012-01-03T13:51:00.000Z","Open":1.30296,"High":1.30298,"Low":1.30281,"Close":1.30287},{"Date":"2012-01-03T13:52:00.000Z","Open":1.30289,"High":1.30309,"Low":1.30289,"Close":1.30295},{"Date":"2012-01-03T13:53:00.000Z","Ope

In [30]:
rates = pd.read_csv('./static/data/data.csv', index_col=False, names=['Date', 'Open', 'High', 'Low', 'Close', 'Spread'], delimiter=' ', skiprows=1380, nrows=10)

In [31]:
print(rates)

                       Date     Open     High      Low    Close   Spread
0  2019-10-21-23:00:00+0000      NaN      NaN      NaN      NaN      NaN
1  2019-10-21-23:01:00+0000  1.11482  1.11482  1.11482  1.11482  0.00023
2  2019-10-21-23:02:00+0000  1.11482  1.11483  1.11481  1.11483  0.00024
3  2019-10-21-23:03:00+0000  1.11484  1.11484  1.11484  1.11484  0.00023
4  2019-10-21-23:04:00+0000  1.11485  1.11485  1.11485  1.11485  0.00023
5  2019-10-21-23:05:00+0000  1.11486  1.11486  1.11486  1.11486  0.00022
6  2019-10-21-23:06:00+0000  1.11485  1.11486  1.11483  1.11486  0.00021
7  2019-10-21-23:07:00+0000  1.11485  1.11485  1.11485  1.11485  0.00020
8  2019-10-21-23:08:00+0000      NaN      NaN      NaN      NaN      NaN
9  2019-10-21-23:09:00+0000      NaN      NaN      NaN      NaN      NaN


In [32]:
del rates['Spread']

In [33]:
rates

,Date,Open,High,Low,Close
0,2019-10-21-23:00:00+0000,NaN,NaN,NaN,NaN
1,2019-10-21-23:01:00+0000,1.11482,1.11482,1.11482,1.11482
2,2019-10-21-23:02:00+0000,1.11482,1.11483,1.11481,1.11483
3,2019-10-21-23:03:00+0000,1.11484,1.11484,1.11484,1.11484
4,2019-10-21-23:04:00+0000,1.11485,1.11485,1.11485,1.11485
5,2019-10-21-23:05:00+0000,1.11486,1.11486,1.11486,1.11486
6,2019-10-21-23:06:00+0000,1.11485,1.11486,1.11483,1.11486
7,2019-10-21-23:07:00+0000,1.11485,1.11485,1.11485,1.11485
8,2019-10-21-23:08:00+0000,NaN,NaN,NaN,NaN
9,2019-10-21-23:09:00+0000,NaN,NaN,NaN,NaN


In [34]:
rates['Date'] = rates['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d-%H:%M:%S%z'))

In [35]:
print(rates)

                       Date     Open     High      Low    Close
0 2019-10-21 23:00:00+00:00      NaN      NaN      NaN      NaN
1 2019-10-21 23:01:00+00:00  1.11482  1.11482  1.11482  1.11482
2 2019-10-21 23:02:00+00:00  1.11482  1.11483  1.11481  1.11483
3 2019-10-21 23:03:00+00:00  1.11484  1.11484  1.11484  1.11484
4 2019-10-21 23:04:00+00:00  1.11485  1.11485  1.11485  1.11485
5 2019-10-21 23:05:00+00:00  1.11486  1.11486  1.11486  1.11486
6 2019-10-21 23:06:00+00:00  1.11485  1.11486  1.11483  1.11486
7 2019-10-21 23:07:00+00:00  1.11485  1.11485  1.11485  1.11485
8 2019-10-21 23:08:00+00:00      NaN      NaN      NaN      NaN
9 2019-10-21 23:09:00+00:00      NaN      NaN      NaN      NaN


In [36]:
rates.to_json(date_format='iso', orient='records')

'[{"Date":"2019-10-21T23:00:00.000Z","Open":null,"High":null,"Low":null,"Close":null},{"Date":"2019-10-21T23:01:00.000Z","Open":1.11482,"High":1.11482,"Low":1.11482,"Close":1.11482},{"Date":"2019-10-21T23:02:00.000Z","Open":1.11482,"High":1.11483,"Low":1.11481,"Close":1.11483},{"Date":"2019-10-21T23:03:00.000Z","Open":1.11484,"High":1.11484,"Low":1.11484,"Close":1.11484},{"Date":"2019-10-21T23:04:00.000Z","Open":1.11485,"High":1.11485,"Low":1.11485,"Close":1.11485},{"Date":"2019-10-21T23:05:00.000Z","Open":1.11486,"High":1.11486,"Low":1.11486,"Close":1.11486},{"Date":"2019-10-21T23:06:00.000Z","Open":1.11485,"High":1.11486,"Low":1.11483,"Close":1.11486},{"Date":"2019-10-21T23:07:00.000Z","Open":1.11485,"High":1.11485,"Low":1.11485,"Close":1.11485},{"Date":"2019-10-21T23:08:00.000Z","Open":null,"High":null,"Low":null,"Close":null},{"Date":"2019-10-21T23:09:00.000Z","Open":null,"High":null,"Low":null,"Close":null}]'

In [27]:
rates['Date'] = rates['Date'].apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d-%H:%M:%S%z'))

In [28]:
print(rates)

                          Date     Open     High      Low    Close
0    2019-10-21 00:00:00+00:00  1.11557  1.11578  1.11541  1.11574
1    2019-10-21 00:01:00+00:00  1.11576  1.11577  1.11570  1.11576
2    2019-10-21 00:02:00+00:00  1.11573  1.11573  1.11566  1.11571
3    2019-10-21 00:03:00+00:00  1.11570  1.11572  1.11569  1.11571
4    2019-10-21 00:04:00+00:00  1.11571  1.11572  1.11570  1.11572
...                        ...      ...      ...      ...      ...
4315 2019-10-23 23:55:00+00:00  1.11309  1.11313  1.11309  1.11312
4316 2019-10-23 23:56:00+00:00  1.11309  1.11309  1.11309  1.11309
4317 2019-10-23 23:57:00+00:00  1.11312  1.11312  1.11310  1.11310
4318 2019-10-23 23:58:00+00:00  1.11312  1.11312  1.11312  1.11312
4319 2019-10-23 23:59:00+00:00  1.11314  1.11314  1.11312  1.11313

[4320 rows x 5 columns]


In [15]:
dt = datetime.datetime.strptime('2019-10-21-00:00:00+0000', '%Y-%m-%d-%H:%M:%S%z')

In [20]:
print(dt.strftime('%Y-%m-%dT%H:%M:%S.000Z'))

2019-10-21T00:00:00.000Z


In [29]:
rates.to_json(date_format='iso', orient='records')

'[{"Date":"2019-10-21T00:00:00.000Z","Open":1.11557,"High":1.11578,"Low":1.11541,"Close":1.11574},{"Date":"2019-10-21T00:01:00.000Z","Open":1.11576,"High":1.11577,"Low":1.1157,"Close":1.11576},{"Date":"2019-10-21T00:02:00.000Z","Open":1.11573,"High":1.11573,"Low":1.11566,"Close":1.11571},{"Date":"2019-10-21T00:03:00.000Z","Open":1.1157,"High":1.11572,"Low":1.11569,"Close":1.11571},{"Date":"2019-10-21T00:04:00.000Z","Open":1.11571,"High":1.11572,"Low":1.1157,"Close":1.11572},{"Date":"2019-10-21T00:05:00.000Z","Open":1.1157,"High":1.1157,"Low":1.11563,"Close":1.11566},{"Date":"2019-10-21T00:06:00.000Z","Open":1.11564,"High":1.11564,"Low":1.11545,"Close":1.1156},{"Date":"2019-10-21T00:07:00.000Z","Open":1.11564,"High":1.11564,"Low":1.11559,"Close":1.11559},{"Date":"2019-10-21T00:08:00.000Z","Open":1.11557,"High":1.1156,"Low":1.11553,"Close":1.1156},{"Date":"2019-10-21T00:09:00.000Z","Open":1.11562,"High":1.11562,"Low":1.11557,"Close":1.11559},{"Date":"2019-10-21T00:10:00.000Z","Open":1.11

In [34]:
dtLimit = datetime.datetime.strptime('2020-03-16T13:13:00Z', '%Y-%m-%dT%H:%M:%SZ')
dtLimit = timezone('UTC').localize(dtLimit)
print(dtLimit.strftime('%Y-%m-%d-%H:%M:%S%z'))

2020-03-16-13:13:00+0000


In [5]:
startDt = datetime.datetime(2018, 12, 31, 10, 50)
startDt = timezone('UTC').localize(startDt)
print(startDt.strftime('%Y-%m-%dT%H:%M:%S.000Z'))
stopDt = datetime.datetime(2018, 12, 31, 11, 0)
stopDt = timezone('UTC').localize(stopDt)
dct = {'startDt': startDt.strftime('%Y-%m-%dT%H:%M:%S.000Z'),
       'stopDt': stopDt.strftime('%Y-%m-%dT%H:%M:%S.000Z'),
       'dir': 1}
print(json.dumps(dct))

2018-12-31T10:50:00.000Z
{"startDt": "2018-12-31T10:50:00.000Z", "stopDt": "2018-12-31T11:00:00.000Z", "dir": 1}


In [11]:
patterns = ''
try:
    with open('./static/data/patterns.json', 'r') as patternFile:
        patterns = json.load(patternFile)
except:
    pass
print(patterns)

[{'startDt': '2018-12-31T10:45:00.000Z', 'stopDt': '2018-12-31T10:58:00.000Z', 'dir': 1}, {'startDt': '2018-12-31T10:50:00.000Z', 'stopDt': '2018-12-31T11:05:00.000Z', 'dir': -1}]


In [12]:
type(patterns)

list

In [13]:
json.dumps(patterns)

'[{"startDt": "2018-12-31T10:45:00.000Z", "stopDt": "2018-12-31T10:58:00.000Z", "dir": 1}, {"startDt": "2018-12-31T10:50:00.000Z", "stopDt": "2018-12-31T11:05:00.000Z", "dir": -1}]'

In [7]:
var = []
print(type(var))
jsoned = json.dumps(var)
print(json.loads(jsoned))

<class 'list'>
[]
